# Horizon Distance Calculation with Mass-Dependent SNR Thresholds

## Find the threshold for BNS systems

* choose 'mass1_source' $\in [1,3]$
* we will use pre-selected injection catalogue data. 
* I will take threshold for individual detector same as that of detector network, for simplicity.
* default sensitivity is consider with O4 psds. 

In [ ]:
# original catalogue data: https://zenodo.org/records/16740117/files/samples-rpo4a_v2_20250503133839UTC-1366933504-23846400.hdf?download=1
# injection_data_bns.json is the reduced data extracted from the above hdf file for testing purpose
! wget https://raw.githubusercontent.com/hemantaph/gwsnr/refs/heads/main/tests/unit/injection_data_bns.json

In [3]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from gwsnr.threshold import SNRThresholdFinder

from gwsnr.utils import get_param_from_json
params = get_param_from_json('injection_data_bns.json')

gstlal_far = params['gstlal_far']
observed_snr_net = params['observed_snr_net']
z = params['z']
mass1_source = params['mass1_source']

test = SNRThresholdFinder(
    catalog_file = None,
    # below are all default values. You can omit them if you want. 
    npool=4,
    original_detection_statistic = dict(
                key_name='gstlal_far',
                threshold=1,  # 1 per year
                parameter=gstlal_far,
            ),
    projected_detection_statistic = dict(
                key_name='observed_snr_net',
                threshold=None, # to be determined
                threshold_search_bounds=(8, 14),
                parameter=observed_snr_net,
            ),
    parameters_to_fit = dict(
        key_name = 'z',
        parameter=z,
    ),
    sample_size=20000,
    selection_range = dict(
        key_name = 'mass1_source',
        range = (1, 3), # in solar masses
        parameter = mass1_source,
    ),
)

best_thr, _, _, _, _ = test.find_threshold(iteration=10, print_output=True, no_multiprocessing=True)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.01s/it]

Best SNR threshold: 11.53


In [8]:
snr_th, snr_th_net = best_thr, best_thr

In [5]:
from gwsnr import GWSNR 

gwsnr = GWSNR(gwsnr_verbose=False)


Initializing GWSNR class...

psds not given. Choosing bilby's default psds
Interpolator will be loaded for L1 detector from ./interpolator_pickle/L1/partialSNR_dict_4.pickle
Interpolator will be loaded for H1 detector from ./interpolator_pickle/H1/partialSNR_dict_4.pickle
Interpolator will be loaded for V1 detector from ./interpolator_pickle/V1/partialSNR_dict_4.pickle




## Horizon Distance - Analytic

In [ ]:
hd_dict_ =  gwsnr.horizon_distance_analytical(mass_1=1.4, mass_2=1.4, snr_th=snr_th)
hd_dict_ # in Mpc

{'L1': array([284.7999009]),
 'H1': array([284.7999009]),
 'V1': array([218.75404729])}

## Horizon Distance - Numerical

In [10]:
hd_dict_, optimal_sky_dict =  gwsnr.horizon_distance_numerical(mass_1=1.4, mass_2=1.4, snr_th=snr_th)

hd_dict_ # in Mpc

{'L1': 328.46671006811084,
 'H1': 328.46671006811084,
 'V1': 252.2944005149766,
 'snr_net': 463.3658527478692}

In [11]:
# optimal sky position
optimal_sky_dict

{'L1': (2.751464995516151, -0.5327666466087082),
 'H1': (2.2513115566737323, -0.8113033727061859),
 'V1': (1.3776350432199378, 0.7615162877515576),
 'snr_net': (2.650441663192078, -0.6782093614739017)}